In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install xlrd
import pandas as pd
data=pd.read_excel("/content/drive/My Drive/BDC/datalatih.xlsx")
data.head()

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,2020-08-17 00:00:00,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,2020-07-17 00:00:00,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,2020-07-13 00:00:00,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,2020-07-08 00:00:00,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,2020-06-24 00:00:00,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg


In [ ]:
# Cek Imbalanced Data
print(data['label'].value_counts())

1    3465
0     766
Name: label, dtype: int64


In [ ]:
df = data[['label', 'narasi']]
df.head()

,label,narasi
0,1,A caller to a radio talk show recently shared ...
1,1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...
2,1,Jokowi adalah presiden terbaik dlm sejarah ban...
3,1,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k..."
4,1,Kadrun kalo lihat foto ini panas dingin . .


In [ ]:
import string, re
def cleansing(df):
  # lower text
  df = df.lower()

  # hapus punctuation
  remove = string.punctuation
  translator=str.maketrans(remove, ' '*len(remove))
  df = df.translate(translator)

  # hapus ASCII dan unicode
  df = df.encode('ascii', 'ignore').decode('utf-8')
  df = re.sub(r'[^\x00-\x7f]',r'', df)

  # hapus angka
  df = re.sub(r'\d+', '', df)

  # hapus link
  df = re.sub(r'http\S+', '', df)

  # hapus whitespace
  df = df.strip()

  # hapus newline 
  df = df.replace('\n', '')
  return df

In [ ]:
narasi1 = []
for index, row in df.iterrows():
  narasi1.append(cleansing(row["narasi"]))

df["narasi"] = narasi1
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,label,narasi
0,1,a caller to a radio talk show recently shared ...
1,1,yth seluruh anggota grup sesuai instruksi gube...
2,1,jokowi adalah presiden terbaik dlm sejarah ban...
3,1,maaf mas dan mbak ini bukan politik tapi ken...
4,1,kadrun kalo lihat foto ini panas dingin


In [ ]:
!pip install Sastrawi

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [ ]:
narasi1=[]
for index, row in df.iterrows():
  narasi1.append(stopword.remove(row["narasi"]))

df["narasi"] = narasi1
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,label,narasi
0,1,a caller to a radio talk show recently shared ...
1,1,yth seluruh anggota grup sesuai instruksi gube...
2,1,jokowi presiden terbaik dlm sejarah bangsa ind...
3,1,maaf mas mbak bukan politik kenyataan pak jo...
4,1,kadrun kalo lihat foto panas dingin


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
narasi1=[]
for index, row in df.iterrows():
  narasi1.append(stemmer.stem(row["narasi"]))

df["narasi"] = narasi1
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,label,narasi
0,1,a caller to a radio talk show recently shared ...
1,1,yth seluruh anggota grup sesuai instruksi gube...
2,1,jokowi presiden baik dlm sejarah bangsa indone...
3,1,maaf mas mbak bukan politik nyata pak jokowi h...
4,1,kadrun kalo lihat foto panas dingin


In [ ]:
n1 = df[df['label']==0].sample(3500, replace=True)
n2 = df[df['label']==1].sample(3500, replace=True)
df = pd.concat([n1, n2])

print(df.shape)
print(df['label'].value_counts(normalize=True))

(7000, 2)
1    0.5
0    0.5
Name: label, dtype: float64


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
X = df.drop('label', axis=1)
y = df.label

In [ ]:
x = X.copy()

In [ ]:
x.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(x)):
    print(i)
    narasi1 = re.sub('[^a-zA-Z]', ' ', x['narasi'][i])
    narasi1 = narasi1.split()
    narasi1 = [ps.stem(word) for word in narasi1 if not word in stopwords.words('indonesian')]
    narasi1 = ' '.join(narasi1)
    corpus.append(narasi1)

Streaming output truncated to the last 5000 lines.
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190

In [ ]:
corpus

['beredarnya foto karangan bunga dikirimkan komisi pemberantasan korupsi tegaskan kpk mengirimkan karangan bunga kpk terafiliasi nama organisasi',
 'kemarin dokter spesiali jantung bernama dr kamariah seminggu pergi perginya menguru ijin praktek idi',
 'isu angkasapura ii klarifikasi execut gener manag bandara internasion soekarno hatta agu haryadi pergantian penambahan nama bandara internasion soekarno hatta soetta termin termin termin bandara',
 'surat beras kpk pastikan surat palsu',
 'ustadz abdul somad dipecat pn pemerintah tindakan rakyat umat islam ustadz perintah',
 'innailaihi wainnailaihi roojiuun ditinggalkan saudara negara muslim dunia arab saudi bentuk negara aliansi strategi milit islam sunni iran indonesia indonesia dianggap merapat komuni china strategi bisni politik ala komuni china syiah iran strategi politik agama syiah yg musuh islam negara islam negara indonesia penduduk islam terbesar dunia dibangga banggakan dianggap umat dunia islam innalillaahi wa inna ilaihi r

In [ ]:
max_features =7000
onehot_repr=[one_hot(words,max_features)for words in corpus] 
onehot_repr

[[2478,
  2325,
  3876,
  6411,
  6455,
  1563,
  5238,
  5378,
  2626,
  3026,
  6060,
  3876,
  6411,
  3026,
  2765,
  2552,
  6116],
 [5433,
  3429,
  6865,
  4653,
  4867,
  497,
  1594,
  524,
  6322,
  3792,
  6720,
  3218,
  2656,
  2030],
 [2425,
  2582,
  5464,
  3749,
  1361,
  4380,
  1598,
  6629,
  5504,
  5441,
  2129,
  6122,
  256,
  564,
  3579,
  2552,
  6629,
  5504,
  5441,
  2129,
  4366,
  4607,
  4607,
  4607,
  6629],
 [4456, 1024, 3026, 1583, 4456, 6422],
 [256, 3746, 1972, 2055, 1761, 4686, 1014, 3479, 2193, 4982, 256, 4920],
 [929,
  6748,
  1699,
  2586,
  423,
  2195,
  4353,
  5854,
  2149,
  2337,
  159,
  2195,
  6089,
  1771,
  5961,
  4982,
  2623,
  469,
  5941,
  5941,
  2584,
  3294,
  1145,
  4890,
  1771,
  3127,
  266,
  1759,
  1145,
  4890,
  2643,
  469,
  1771,
  266,
  199,
  2643,
  3384,
  2919,
  4982,
  2195,
  4982,
  2195,
  5941,
  3193,
  4982,
  940,
  5854,
  5783,
  6094,
  2584,
  2193,
  5854,
  4982,
  6440,
  2434,
  5533,
  

In [ ]:
sent_length=70
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2765 2552 6116]
 [   0    0    0 ... 3218 2656 2030]
 [   0    0    0 ... 4607 4607 6629]
 ...
 [   0    0    0 ... 4464 2013 4589]
 [   0    0    0 ... 6601  272 2414]
 [   0    0    0 ... 6100 1384 4185]]


In [ ]:
max_features = 7000
embed_dim = 150
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 70, 150)           1050000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               100400    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 1,150,501
Trainable params: 1,150,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(7000, (7000,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [ ]:
model.fit(X_train,y_train, validation_split=0.3, epochs=5, batch_size=2)

Epoch 1/5
1715/1715 [==============================] - 85s 50ms/step - loss: 0.4946 - accuracy: 0.7405 - val_loss: 0.3068 - val_accuracy: 0.8762
Epoch 2/5
1715/1715 [==============================] - 83s 48ms/step - loss: 0.0936 - accuracy: 0.9653 - val_loss: 0.2805 - val_accuracy: 0.8918
Epoch 3/5
1715/1715 [==============================] - 81s 47ms/step - loss: 0.0216 - accuracy: 0.9918 - val_loss: 0.4038 - val_accuracy: 0.8946
Epoch 4/5
1715/1715 [==============================] - 83s 48ms/step - loss: 0.0083 - accuracy: 0.9965 - val_loss: 0.5305 - val_accuracy: 0.8959
Epoch 5/5
1715/1715 [==============================] - 84s 49ms/step - loss: 0.0054 - accuracy: 0.9977 - val_loss: 0.5446 - val_accuracy: 0.8925


In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1012,   69],
       [ 122,  897]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.909047619047619

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1081
           1       0.93      0.88      0.90      1019

    accuracy                           0.91      2100
   macro avg       0.91      0.91      0.91      2100
weighted avg       0.91      0.91      0.91      2100



Memprediksi Data Uji

In [ ]:
!pip install xlrd
import pandas as pd
datauji=pd.read_excel("/content/drive/My Drive/BDC/datauji.xlsx")
datauji.head()

,ID,tanggal,judul,narasi,nama file gambar
0,238057,2020-07-13 00:00:00,Narasi Tito Karnavian Berideologi Komunis Kare...,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...,238057.jpg
1,238158,2020-07-06 00:00:00,Anies: Seberat beratnya Pekerjaan Akan terasa ...,Seberat beratnya Pekerjaan Akan terasa ringan ...,238158.jpg
2,238865,2020-04-22 00:00:00,Hindu di india Melemparkan Patung Buatan Merek...,Hindu di india melemparkan patung buatan merek...,238865.jpg
3,248298,2019-10-22 00:00:00,RSCM Praktekkan Penyedotan Plug Vena/Saluran ...,Mulai Hari ini di RSCM mulai diPraktekkan Peny...,248298.jpg
4,255176,2020-05-01 00:00:00,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran...",255176.jpg


In [ ]:
df_uji = datauji[['narasi']]
df_uji.head()

,narasi
0,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA...
1,Seberat beratnya Pekerjaan Akan terasa ringan ...
2,Hindu di india melemparkan patung buatan merek...
3,Mulai Hari ini di RSCM mulai diPraktekkan Peny...
4,"Untuk sekedar info, Bagi anda yg punya ansuran..."


In [ ]:
narasi_uji = []
for index, row in df_uji.iterrows():
  narasi_uji.append(cleansing(row["narasi"]))

df_uji["narasi"] = narasi_uji
df_uji.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,narasi
0,tito karnivan itu beridiologi komunis dia bisa...
1,seberat beratnya pekerjaan akan terasa ringan ...
2,hindu di india melemparkan patung buatan merek...
3,mulai hari ini di rscm mulai dipraktekkan peny...
4,untuk sekedar info bagi anda yg punya ansuran...


In [ ]:
narasi_uji=[]
for index, row in df_uji.iterrows():
  narasi_uji.append(stopword.remove(row["narasi"]))

df_uji["narasi"] = narasi_uji
df_uji.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,narasi
0,tito karnivan beridiologi komunis bisa katakan...
1,seberat beratnya pekerjaan terasa ringan bila ...
2,hindu india melemparkan patung buatan ke laut ...
3,mulai hari rscm mulai dipraktekkan penyedotan ...
4,sekedar info yg punya ansuran cicilan belum k...


In [ ]:
narasi_uji=[]
for index, row in df_uji.iterrows():
  narasi_uji.append(stemmer.stem(row["narasi"]))

df_uji["narasi"] = narasi_uji
df_uji.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,narasi
0,tito karnivan beridiologi komunis bisa kata pk...
1,berat berat kerja asa ringan bila di kerja
2,hindu india lempar patung buat ke laut apa lak...
3,mulai hari rscm mulai praktek sedot plug sumba...
4,dar info yg punya ansuran cicil belum longgar ...


In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus_uji = []
for i in range(0, len(df_uji)):
    print(i)
    narasi_uji1 = re.sub('[^a-zA-Z]', ' ', df_uji['narasi'][i])
    narasi_uji1 = narasi_uji1.split()
    narasi_uji1 = [ps.stem(word) for word in narasi_uji1 if not word in stopwords.words('indonesian')]
    narasi_uji1 = ' '.join(narasi_uji1)
    corpus_uji.append(narasi_uji1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
corpus

['kabar kader partai karya pollycarpu budihari prijanto mantan pidana bunuh aktivi hak asasi manusia ham munir said thalib bantah',
 'kawal surat al hujurat lawan hoax',
 'bahaya makan rambutan minum teh mani sumbat salur napa',
 'lsf klarifikasi kait potong menit durasi film aveng infin war',
 'yg nyinyirin pendeta poligami',
 'tokoh dukung capr kena motif',
 'cek video',
 'seriu negeri mayorita muslim pidana kumpul kebo alia zina lgbt langgar hukumkatanya ideolog pancasila zina lgbt sesuai pancasila pancasila lahir islam islam kera haram zina lgbtkalau pancasila legal zina lgbt arti pancasila islam anti zina anti lgbt dikatain anti pancasila anti nkri seriu nanyakomen yg',
 'akibat candu main game hp akibatx sesal gak yg tanam it yg tuai sayang anakkita jng main hp akibatx fatal gararadiasi',
 'lapor ndan koq undang proyek ground break ka cepat bahasa cina jakarta salah propinsi cina gagal wara',
 'menteri agama republlik indonesiamemberikan ijin operasion selenggara ibadah haji umra

In [ ]:
onehot_repr_uji=[one_hot(words,max_features)for words in corpus_uji] 
onehot_repr_uji

[[6436, 5063, 2533, 5349, 4742, 1532, 3719, 5349, 4228, 3707],
 [307, 307, 6369, 5053, 1658, 6369],
 [6901,
  6005,
  3764,
  2746,
  2960,
  6809,
  3990,
  3760,
  2808,
  5808,
  1264,
  2479,
  3170,
  3334,
  3032],
 [6746,
  2879,
  3136,
  1501,
  2475,
  1007,
  4407,
  4748,
  5997,
  6014,
  4308,
  4531,
  3002,
  2948,
  6613,
  463,
  4765,
  5281,
  1996,
  4559,
  2475,
  5997,
  6095,
  1394,
  4308,
  4079],
 [2984,
  4348,
  1996,
  5258,
  6487,
  4718,
  3613,
  4120,
  1392,
  5471,
  4565,
  6055,
  872,
  3045,
  5069,
  2237,
  2992,
  5471,
  2240,
  1392,
  5471,
  563,
  3861,
  5471,
  1392,
  1905,
  3676,
  2479,
  4718,
  2159,
  5258,
  6487,
  4210,
  5642,
  6756,
  1325,
  3166,
  222,
  5878,
  2808,
  2287,
  4901,
  2022,
  3195],
 [932, 1478, 5030, 6453, 4465, 6908, 6453, 6773, 554, 879],
 [1776,
  5563,
  583,
  4465,
  5240,
  5557,
  3657,
  5568,
  1897,
  3702,
  1283,
  3195,
  6702,
  6702,
  1776,
  5344,
  5988,
  497,
  6843,
  4181,
  5

In [ ]:
sent_length_uji=70
embedded_docs_uji=pad_sequences(onehot_repr_uji,padding='pre',maxlen=sent_length_uji)
print(embedded_docs_uji)

[[   0    0    0 ... 5349 4228 3707]
 [   0    0    0 ... 5053 1658 6369]
 [   0    0    0 ... 3170 3334 3032]
 ...
 [   0    0    0 ... 1898 5683  191]
 [   0    0    0 ...  381 2346  825]
 [   0    0    0 ... 5405 4411 2352]]


In [ ]:
X_uji = np.array(embedded_docs_uji)

In [ ]:
prediksi = model.predict_classes(X_uji)

In [ ]:
print(prediksi)

[[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]


In [ ]:
A = datauji['ID']

In [ ]:
B = pd.DataFrame(data=prediksi, columns=["prediksi"])

In [ ]:
Prediksi = pd.concat([A, B], axis=1)
Prediksi.head()

,ID,prediksi
0,238057,0
1,238158,1
2,238865,1
3,248298,1
4,255176,1


In [ ]:
Prediksi.to_csv("/content/drive/My Drive/BDC/Prediksi.csv")